In [2]:
import pandas as pd
df = pd.read_csv('imdb_indonesian_movies_2.csv')
df = df.drop(columns=['judul_film'])

In [6]:
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


In [7]:
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


In [8]:
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values
print(sinopsis)
print(label)

['Raden Mas Said putra sulung Tumenggung Wilarikta di bawah Kerajaan Majapahit yang berkuasa di wilayah Tuban melihat sekeluarga miskin yang menderita busung lapar. Ia merasa sangat prihatin dan hati nuraninya tergugah untuk menolong. Kemudian ia mencoba secara diam-diam mengambil makanan dari lumbung orang tuanya. Perbuatan itu tidak disetujui orang tuanya bahkan ia dihukum sekap di gudang makanan itu. Sejak kejadian itu RM Said yang tumbuh dewasa tidak betah tinggal di rumah. Ia berkelana dari daerah satu ke daerah lainnya. Dari sanalah ia tahu betapa banyak penyelewengan dan kesewenang-wenangan para lurah yang munafik. Mereka selalu mengkambing-hitamkan Tumenggung untuk menutupi kejahatannya. Atas laporan RM Said ayahnya kemudian sadar. Tetapi kemudian ia dianggap sebagai sumber fitnah. Dalam kelananya kemudian ia bertemu dengan Sunan Bonang yang banyak mencurahkan ilmunya kepada RM Said. Ia pun kemudian melakukan tapa di pinggir kali. Berkat ketabahannya menghadapi berbagai cobaan 

In [9]:
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_latih) 
tokenizer.fit_on_texts(sinopsis_test)
 
print("sekuens latih :")
sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
print(sekuens_latih)

print("sekuens test :")
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)
print(sekuens_test)

print("padded latih :")
padded_latih = pad_sequences(sekuens_latih) 
print(padded_latih)

print("padded test :")
padded_test = pad_sequences(sekuens_test)
print(padded_test)

sekuens latih :
[[422, 4275, 1740, 4667, 4667, 160, 30, 4667, 4275, 2, 72, 3, 236, 699, 11, 75, 106, 111, 21, 4276, 2501, 2923, 48, 164, 2, 1939, 202, 18, 631, 31, 132, 168, 13, 665, 259, 4667, 4275, 242, 1740, 7, 44, 2924, 31, 292, 2502, 9, 1104, 263, 2925, 263, 1740, 4277, 2, 133, 276, 1740, 550, 435, 31, 4277, 72, 2503, 4276, 1740, 286, 252, 2926, 1940, 4278, 2, 44, 1302, 4667, 62, 52, 4667, 4667, 2202, 4667, 2, 3484, 3485, 2927, 4277, 3, 273, 4276], [1184, 3, 2504, 1020, 35, 17, 195, 1184, 20, 12, 960, 2, 4279, 4, 3486, 1303, 4667, 1564, 20, 16, 1304, 422, 490, 79, 2504, 19, 20, 1420, 1304, 14, 7, 34, 98, 3487, 403, 7, 113, 132, 961, 33, 4667, 4667, 1304, 4667, 8, 90, 4667, 423, 7, 20, 4667, 4667, 1021, 80, 3, 13, 194, 194, 4667, 2504, 102, 20, 2, 63, 2203, 2, 2204, 2505, 21, 1184, 33, 1741, 2506, 259, 80, 4, 4667, 9, 2507, 4280, 1184, 379, 1742, 3, 286, 4667, 81, 11, 129, 244, 2507, 9, 30, 666, 1305, 23, 161, 3488, 3, 213, 81, 1185, 207, 2203, 2, 500, 1186, 40, 1941, 305, 435, 118

In [12]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [13]:
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 13s - loss: 1.6100 - accuracy: 0.1828 - val_loss: 1.6100 - val_accuracy: 0.1940
Epoch 2/30
26/26 - 10s - loss: 1.6016 - accuracy: 0.2823 - val_loss: 1.6251 - val_accuracy: 0.2438
Epoch 3/30
26/26 - 10s - loss: 1.5276 - accuracy: 0.3545 - val_loss: 1.5693 - val_accuracy: 0.3035
Epoch 4/30
26/26 - 10s - loss: 1.2484 - accuracy: 0.4378 - val_loss: 1.7753 - val_accuracy: 0.3333
Epoch 5/30
26/26 - 10s - loss: 1.0446 - accuracy: 0.5510 - val_loss: 1.8353 - val_accuracy: 0.3184
Epoch 6/30
26/26 - 10s - loss: 0.8170 - accuracy: 0.6430 - val_loss: 2.2254 - val_accuracy: 0.2786
Epoch 7/30
26/26 - 10s - loss: 0.6965 - accuracy: 0.7575 - val_loss: 2.3711 - val_accuracy: 0.2935
Epoch 8/30
26/26 - 10s - loss: 0.4796 - accuracy: 0.8321 - val_loss: 2.9187 - val_accuracy: 0.3035
Epoch 9/30
26/26 - 11s - loss: 0.3340 - accuracy: 0.9005 - val_loss: 2.7312 - val_accuracy: 0.3532
Epoch 10/30
26/26 - 10s - loss: 0.2488 - accuracy: 0.9428 - val_loss: 3.4570 - val_accuracy: 0.3333
Epoch 11/